In [28]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import pathlib
import swifter
from functools import reduce

from shapely import wkt
from shapely.geometry import Point, Polygon, LineString, MultiPoint

import geojson
import h3

from geovoronoi import voronoi_regions_from_coords
crs4326 = {'init': 'epsg:4326'}

In [84]:
DIR_ROOT = os.path.join(pathlib.Path().absolute(), '../../..' )

influences_df = pd.read_csv(DIR_ROOT + '/data/04_itinerary/influences.csv')
influences_df = influences_df[influences_df['domestic'] == True]

localities = pd.read_csv(DIR_ROOT + '/data/02_processed/localities.csv')
localities['geometry'] = localities['geometry'].apply(wkt.loads)
localities = gpd.GeoDataFrame(localities, crs=crs4326).set_geometry('geometry')

In [106]:
bbox = localities.buffer(1).total_bounds
bounds = dict(geojson.Polygon([[[bbox[0], bbox[1]], [bbox[0], bbox[3]], [bbox[2], bbox[3]], [bbox[2], bbox[1]]]]))
hex_ids = list(h3.polyfill(bounds, 4))

hex_df = gpd.GeoDataFrame(
    [{"geometry": Polygon(h3.h3_to_geo_boundary(x)), "id": x} for x in hex_ids], crs=crs4326
)
hex_df.set_index('id', inplace=True)

hex_df['id'] = hex_df.index
hex_df['geometry'] = hex_df.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x['id'])), axis=1)
hex_df

<ipython-input-106-7dfe25777e9d>:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bbox = localities.buffer(1).total_bounds


,geometry,id
id,,
84535a9ffffffff,"POLYGON ((21.81184 47.94434, 21.60953 47.81349...",84535a9ffffffff
8452cedffffffff,"POLYGON ((11.57349 46.68279, 11.37786 46.56439...",8452cedffffffff
8462a47ffffffff,"POLYGON ((8.16659 54.60426, 8.34806 54.65809, ...",8462a47ffffffff
84525c1ffffffff,"POLYGON ((16.29343 50.25178, 16.09898 50.12746...",84525c1ffffffff
8452e2dffffffff,"POLYGON ((14.52796 44.80263, 14.32498 44.68153...",8452e2dffffffff
...,...,...
8463ae7ffffffff,"POLYGON ((12.67742 53.61361, 12.49364 53.49419...",8463ae7ffffffff
8463907ffffffff,"POLYGON ((9.45919 50.37061, 9.64328 50.43161, ...",8463907ffffffff
8452d8bffffffff,"POLYGON ((10.02907 45.95947, 9.83472 45.84298,...",8452d8bffffffff


In [115]:
def get_influence_origins(hex_geo):
    localities_in_hex = localities[localities.intersects(hex_geo)]

    infl_list = []
    for lihi, locality_in_hex in localities_in_hex.iterrows():
        inf_object_rows = influences_df[influences_df['name'] == locality_in_hex['name']]
        infl_list.extend(inf_object_rows['locality'].tolist())

    return { x: infl_list.count(x) for x in infl_list}

hex_df['infl'] = hex_df.apply(
    lambda x: get_influence_origins(x['geometry']), 
    axis=1
)
print(hex_df['infl'].tolist())

hex_df['all'] = hex_df.apply(
    lambda x: sum(x['infl'].values()), 
    axis=1
)

centers_to_check = ['Praha', 'Brno', 'Wien', 'Znojmo', 'Graz', 'Krems an der Donau', 'Linz']


for center in centers_to_check:
    hex_df[center] = hex_df.apply(
        lambda x: x['infl'][center] if center in x['infl'] else 0, 
        axis=1
    )

hex_df['others'] = hex_df.apply(
    lambda x: x['all'] - sum(x[center] for center in centers_to_check) , 
    axis=1
)



[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Korneuburg': 1, 'Wels': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Brno': 4, 'Wien': 1, 'Olomouc': 1, 'Plasy': 1, 'Uherské Hradiště': 1}, {}, {}, {}, {}, {}, {'Praha': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Freistadt': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Havlíčkův Brod': 2, 'Poděbrady': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {'Graz': 3, 'Praha': 4, 'Wien': 9, 'Velehrad': 2, 'Wiener Neustadt': 3, 'Poděbrady': 1, 'Znojmo': 4, 'Sadská': 1, 'Zbraslav': 1, 'Brno': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Graz': 1, 'Wiener Neustadt': 1, 'Velehrad': 1, 'Praha': 1}, {}, {}, {}, {'Brno': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {

In [116]:

hex_df.reset_index(drop=True).to_file(DIR_ROOT + '/data/06_outputs/hex_influence.geojson', driver="GeoJSON")

hex_df_geo = hex_df.copy()
hex_df_geo['centroid'] = hex_df_geo.apply(
    lambda x: x['geometry'].centroid, 
    axis=1
)
hex_df_geo = hex_df_geo.drop(columns=["geometry"])
hex_df_geo.set_geometry('centroid', inplace=True)
hex_df_geo = hex_df_geo.reset_index(drop=True)
hex_df_geo.to_file(DIR_ROOT + '/data/06_outputs/hex-centroids_influence.geojson', driver="GeoJSON")
